In [14]:
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt 
import pickle
import tensorly
import matplotlib.ticker as ticker
import tensorsketch
from tensorsketch.tensor_approx import TensorApprox
import warnings
warnings.filterwarnings('ignore')
import scipy.io
from tensorsketch.sketch import Sketch

In [15]:
# data_idx = ['1000', '1010','1020','1030','1043', '1051','1060','1070','1080','1090','1100','1121','1131','1140','1150','1162','1171','1180','1192','1200']
# data_name = ['data/data_'+idx+'E-02.mat' for idx in data_idx]
# pressure = [scipy.io.loadmat(name).get('data')[:,:,:,3] for name in data_name]
# pressure = np.stack(pressure,axis=3)
# np.save("data/pressure.npy",pressure)

In [16]:
pressure = np.load("data/pressure.npy")

In [23]:
def update_sketch(arm_sketches, core_sketch, data, ks, ss, rm_typ = 'g', ts_dim = 0, typ = 'turnstile', seed = 1):
    if typ == 'turnstile': 
        new_sketch = tensorsketch.sketch.Sketch(data, ks, seed, ss)
        [new_arm_sketches, new_core_sketch] = new_sketch.get_sketches()
        for i in range(len(arm_sketches)): 
            arm_sketches[i] = arm_sketches[i] + new_arm_sketches[i]
        core_sketch = core_sketch + new_core_sketch
        # print(new_core_sketch)
        # print(new_arm_sketches)
        return [arm_sketches, core_sketch]
    elif typ == "ts": 
        pass

In [24]:
pressure.shape

(1408, 128, 128, 20)

In [29]:
ks = [352,32,32,10]
ss = [704, 64, 64, 15]
ranks = [200, 20, 20, 5]

In [26]:
def update_combustion(data, ks, ss, rm_typ = 'g', seed = 1):    
    np.random.seed(seed)
    # Initialize the arm sketches and core sketch 
    arms_sketches = [np.zeros([length,ks[i]]) for i, length in enumerate(list(data.shape))]
    core_sketch = np.zeros(ss)
    for i in np.arange(data.shape[3]): 
        print(i)
        new_data = np.zeros(data.shape)
        new_data[:,:,:,i] = data[:,:,:,i]
        sketches = update_sketch(arms_sketches, core_sketch, new_data, ks, ss, rm_typ= rm_typ, seed =  seed)
    return [arms_sketches, core_sketch] 
        

In [27]:
test = update_combustion(pressure[:,:,:,:5], ks, ss)

0


Using numpy backend.


1


Using numpy backend.


2


Using numpy backend.


3


Using numpy backend.


4


Using numpy backend.


In [ ]:
def combustion_sim(data, ranks, ks, ss, seed = 1, rm_typ = 'g'):
    [arm_sketches, core_sketch] = update_combustion(data, ks, ss)
    tapprox = tensorsketch.tensor_approx.TensorApprox(data, ranks = ranks, ks = ks, \
            ss = ss, random_seed = seed, rm_typ = rm_typ, store_phis = False)
    
    _, _, _, hooi_rerr, _ = tapprox.tensor_approx('hooi')
    _, _, _, two_pass_rerr, _ = tapprox.tensor_approx('twopass')
    _, _, _, one_pass_rerr, _ = tapprox.tensor_approx('onepass')
    result = [hooi_result, two_pass_result, one_pass_result] 
    print(result)
    return result

In [33]:
seed = 1; rm_typ = 'g'

In [43]:
data = pressure[:,:,:,:5]

In [44]:
pressure.shape

(1408, 128, 128, 20)

In [45]:
ks = [564,52,52,3]
ss = [1100, 100, 100, 4]
ranks = [282, 26, 26,2]

In [46]:
tapprox = tensorsketch.tensor_approx.TensorApprox(data, ranks = ranks, ks = ks, \
            ss = ss, random_seed = seed, rm_typ = rm_typ, store_phis = False)
    
_, _, _, hooi_rerr, _ = tapprox.tensor_approx('hooi')
print(hooi_rerr)

_, _, _, two_pass_rerr, _ = tapprox.tensor_approx('twopass')
print(two_pass_rerr)

_, _, _, one_pass_rerr, _ = tapprox.tensor_approx('onepass')
print(one_pass_rerr)

Using numpy backend.


0.48093689592610483


Using numpy backend.
Using numpy backend.


0.7420139824068969


Using numpy backend.


KeyboardInterrupt: 